In [ ]:
# # Read the contents of the original file with explicit encoding
# with open('course.txt', 'r', encoding='utf-8') as file:
#     lines = file.readlines()

# values_dicts = []
# keys = []
# for line in lines:
#     key, values = line.split(':',1)
#     key = key.strip()
#     # print(key)
#     values_dict = eval(values)
#     # print(values_dict)
#     values_dicts.append(values_dict)
#     keys.append(key)


# import pandas as pd
# df = pd.DataFrame(values_dicts, index = keys)
# df = df.rename_axis('course_id')
# df = df.reset_index()
# df.to_csv('courses.csv', index=False)

In [1]:
import pandas as pd
df = pd.read_csv("courses.csv")
df.head()

,course_id,course_title,training_organisation,uen,full_fee,no_of_respondents,course_attendee_count,course_objective,course_content,relevant_job_roles,entry_requirement,training_commitment,duration_hour,url,area_of_training
0,TGS-2023019838,Real Estate Salesperson (RES) Course,SEAA,T15SS0188B,$768.52,335.0,1053.0,The objectives of the RES Examination are to e...,The RES course prepares new entrants with Real...,NaN,Singapore Citizen & SPR 21 years old above,Full Time,54.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
1,TGS-2020501266,Value Investing Programme (Interpretation of F...,MIND KINESIS INVESTMENTS PTE. LTD.,201202566W,$500.00,139.0,1103.0,Value Investing was founded and taught by Prof...,1) Basics of Securities Analysis\n2) Interpret...,NaN,"O Level, A Level, Diploma",Part Time,13.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
2,TGS-2022016789,User Experience Design Bootcamp,VERTICAL INSTITUTE PTE. LTD.,201923625H,"$1,650.00",183.0,596.0,-Have a comprehensive understanding of UX desi...,Vertical Institute’s User Experience Design Bo...,NaN,This is a beginner-friendly program with no pr...,Part Time,28.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
3,TGS-2023022106,Cybersecurity Bootcamp,VERTICAL INSTITUTE PTE. LTD.,201923625H,"$1,650.00",167.0,521.0,-Understand the basic of cyber-security and th...,Vertical Institute’s Cybersecurity Bootcamp la...,NaN,This is a beginner-friendly program with no pr...,Part Time,28.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"
4,TGS-2020000386,Graduate Certificate in Digital Finance Module...,SINGAPORE MANAGEMENT UNIVERSITY,200000267Z,"$1,600.00",102.0,600.0,"At the end of the 2-day module, participants w...",The module focuses on the fundamentals of the ...,NaN,NaN,NaN,17.00 hours,https://www.myskillsfuture.gov.sg/content/port...,"Accountancy, Banking & Finance"


In [2]:
from langchain_community.document_loaders.csv_loader import CSVLoader
loader = CSVLoader(file_path = "courses.csv", encoding='utf8')
data = loader.load()


In [3]:
import transformers
import torch
from transformers import (
  AutoTokenizer, 
  AutoModelForCausalLM, 
  BitsAndBytesConfig,
  pipeline
)

from langchain_text_splitters  import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

from langchain.prompts import PromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain


MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.2'
##############################################
#TOKENIZER 
##############################################
model_config =transformers.AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#####################################
#BitsAndBytes parameters
#####################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

#################################################################
# Set up quantization config
#################################################################
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

In [5]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [6]:
#################################################################
# Load pre-trained config
#################################################################
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
)


def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(model))

text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

text_generation_pipeline2 = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=1000,
)

mistral_llm2 = HuggingFacePipeline(pipeline = text_generation_pipeline2)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


bin c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\bitsandbytes\libbitsandbytes_cuda118.dll


Loading checkpoint shards: 100%|██████████| 3/3 [00:06<00:00,  2.29s/it]
You are calling `save_pretrained` to a 4-bit converted model, but your `bitsandbytes` version doesn't support it. If you want to save 4-bit models, make sure to have `bitsandbytes>=0.41.3` installed.


trainable model parameters: 262410240
all model parameters: 3752071168
percentage of trainable model parameters: 6.99%


In [7]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2048,
    chunk_overlap = 256
)

chunked_data = text_splitter.split_documents(data)

# load chunked data into the FAISS index
db = FAISS.from_documents(chunked_data,
                          HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))

retriever = db.as_retriever()

In [8]:
# creating prompt template
prompt_template = """
### [INST]

You are restricted to retrieve relevant courses from the provided context.
Your role is a professional course consultant and career advisor.
Provide suitable skills future courses from the provided context.

Never reply with your own existing knowledge, always use the knowledge from the context
Do not come up with your own creation. Output the original courses. 
Do not mention things that you are unsure or do not know, if encounter, politely apologize.

{context}

[/INST]

"""
prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template,
)

llm_chain = LLMChain(llm=mistral_llm, prompt=prompt, output_key = "output1")

prompt_template2 = """
[INST]
Based on {output1}, DO NOT change the title and the relevant informations. Be as factual as possible, explain the rationale to taking these courses based on {question}.
Do not mention things that you are unsure about or do not know. Your role here is a professional career advisor. Use tone that suits your role
[/INST]
"""
prompt2 = PromptTemplate(
    input_variables=["output1","question"],
    template=prompt_template2
)
llm_chain2 = LLMChain(llm =mistral_llm2, prompt=prompt2, output_key="output2")

from langchain.chains import SequentialChain
overall_chain = SequentialChain(
    chains=[llm_chain, llm_chain2],
    input_variables=["question", "context"],
    output_variables=["output2"],
    verbose=True
)

rag_chain = (
    {"context":retriever, "question":RunnablePassthrough()}
    | overall_chain
)

rag_chain.invoke("I am a experience software developer. Now I wish to start my own business. What courses do you recommend?")['output2']



> Entering new SequentialChain chain...


c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\transformers\models\mistral\modeling_mistral.py:688: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\user\Desktop\AiSingapore24\.venv\Lib\site-packages\transformers\generation\configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=


> Finished chain.


'As a professional career advisor, based on your background as an experienced software developer and your aspiration to start your own business, I would recommend the following courses to enhance your entrepreneurial and business management skills:\n\n1. Entrepreneurial Strategies for Business Management - Part of Certificate in Business Management (DB9AM1)(Classroom) by Ngee Ann Polytechnic\n   This course introduces new product and service ideation, design thinking, and lean start-up technology, which can be valuable for someone transitioning from a technical role to entrepreneurship. It also helps learners manage risks and generate innovative ideas to capture new market opportunities.\n\n2. Modular Certificate in Business Management - Part of Diploma in Business Practice (Administration & Management) (DBAM3) by Ngee Ann Polytechnic\n   This comprehensive program covers the principles and concepts of key business functions, human resource management, organizational culture, and integ